In [ ]:
#Prompt(Chatgpt 4o): How to calculate FID and Clip score of trained emoji generator model

In [1]:
# Imports
import torch
from diffusers import StableDiffusionPipeline
from peft import PeftModel
from PIL import Image
import os
import pandas as pd
from tqdm import tqdm
from transformers import CLIPProcessor, CLIPModel

In [2]:
# Setup
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

# Prompts for generating stickers
PROMPTS = [
    "A happy cat holding a tiny heart",
    "Cool panda wearing sunglasses and headphones",
    "Cute avocado giving a thumbs up",
    "Sleepy moon with a pillow and stars",
    "Dancing banana with sparkles around",
    "Angry chili pepper blowing fire",
    "Smiling robot with a peace sign",
    "Laughing unicorn with rainbow hair",
    "Cute ghost saying boo with sparkles",
    "Excited frog jumping with joy",
    "Shy mushroom hiding behind leaves",
    "Cute sushi roll with rosy cheeks",
    "Cheerful sun waving hello",
    "Sad teardrop emoji with big eyes",
    "Boba tea with happy face and straw",
    "High-fiving stars with happy faces",
    "Crying cupcake with melting frosting",
    "Cool dog skateboarding",
    "Excited duck throwing confetti",
    "Cute alien giving a thumbs up",
    "Happy watermelon slice with smile",
    "Surprised bunny with wide eyes",
    "Tired coffee cup with droopy eyes",
    "Excited bee flying with sparkles",
    "Winking taco with salsa hat",
    "Angry broccoli flexing muscles",
    "Cute penguin with scarf and mittens",
    "Peaceful cloud meditating",
    "Jumping cookie saying 'Yay!'",
    "Nervous donut biting its lip"
]

# Output directory
save_dir = "outputs/Sticker_manual_prompt_clip_only"
os.makedirs(save_dir, exist_ok=True)

# Load Stable Diffusion with LoRA
model_id = "sd-legacy/stable-diffusion-v1-5"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16).to(device)
pipe.unet = PeftModel.from_pretrained(pipe.unet, "../evaluation/sticker_diffusion_qlora/final_model")
pipe.enable_attention_slicing()
pipe.safety_checker = lambda images, clip_input, **kwargs: (images, [False] * len(images))
print(" LoRA model loaded and ready.")

# Generate images
results = []
for i, prompt in enumerate(tqdm(PROMPTS, desc="Generating images")):
    result = pipe(prompt, num_inference_steps=50, guidance_scale=7.5, height=256, width=256)
    image = result.images[0]
    image_path = os.path.join(save_dir, f"sticker_{i:02}.png")
    image.save(image_path)
    results.append({"prompt": prompt, "generated_image_path": image_path})

results_df = pd.DataFrame(results)

# Load CLIP model
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").eval().to(device)
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Compute CLIP scores
clip_scores = []
for _, row in tqdm(results_df.iterrows(), total=len(results_df), desc="Calculating CLIP scores"):
    image = Image.open(row["generated_image_path"]).convert("RGB")
    inputs = clip_processor(text=row["prompt"], images=image, return_tensors="pt", padding=True).to(device)
    with torch.no_grad():
        # Run processor
        inputs = clip_processor(text=row["prompt"], images=image, return_tensors="pt", padding=True).to(device)

        # Extract pixel and text features separately
        image_inputs = {"pixel_values": inputs["pixel_values"]}
        text_inputs = {
            "input_ids": inputs["input_ids"],
            "attention_mask": inputs["attention_mask"]
        }

        # Get features
        image_embeds = clip_model.get_image_features(**image_inputs)
        text_embeds = clip_model.get_text_features(**text_inputs)

        # Normalize and compute cosine similarity
        image_embeds = image_embeds / image_embeds.norm(dim=-1, keepdim=True)
        text_embeds = text_embeds / text_embeds.norm(dim=-1, keepdim=True)

        score = torch.nn.functional.cosine_similarity(image_embeds, text_embeds).item()
        clip_scores.append(score)

results_df["clip_score"] = clip_scores
results_df.to_parquet(os.path.join(save_dir, "clip_scores.parquet"), index=False)

# Print summary
print(f"\n All done! Results saved to: {save_dir}")
print(f" Average CLIP Score: {results_df['clip_score'].mean():.4f}")

Using device: cuda


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

 LoRA model loaded and ready.


Generating images:   0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

Generating images:  97%|█████████▋| 29/30 [02:55<00:06,  6.27s/it]

  0%|          | 0/50 [00:00<?, ?it/s]

Calculating CLIP scores: 100%|██████████| 30/30 [00:00<00:00, 30.86it/s]


 All done! Results saved to: outputs/Sticker_manual_prompt_clip_only
 Average CLIP Score: 0.3070


In [5]:
import torch
import os
import pandas as pd
from torch.utils.data import DataLoader, random_split
from torchvision import transforms
from torchvision.datasets.folder import default_loader
from tqdm import tqdm
from diffusers import StableDiffusionPipeline
from peft import PeftModel
from PIL import Image
from pytorch_fid.fid_score import calculate_fid_given_paths

In [6]:
# Setup
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

# Load validation dataset
df = pd.read_parquet("../data/processed_sticker_dataset.parquet")

class EmojiDataset(torch.utils.data.Dataset):
    def __init__(self, df):
        self.df = df
        self.transform = transforms.Compose([
            transforms.Resize((256, 256)),
            transforms.ToTensor()
        ])

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        image_tensor = torch.load(self.df.iloc[idx]['image_path']).float()
        prompt = self.df.iloc[idx]['prompt']
        return {"pixel_values": image_tensor, "prompt": prompt}

dataset = EmojiDataset(df)
train_size = int(0.99 * len(dataset))
_, val_set = random_split(dataset, [train_size, len(dataset) - train_size])

# Generate images from validation prompts
model_id = "sd-legacy/stable-diffusion-v1-5"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16).to(device)
pipe.unet = PeftModel.from_pretrained(pipe.unet, "../evaluation/sticker_diffusion_qlora/final_model")
pipe.enable_attention_slicing()
pipe.safety_checker = lambda images, clip_input, **kwargs: (images, [False] * len(images))
print(" LoRA model loaded.")

# Output directories for FID
generated_dir = "outputs/sticker_fid/generated"
real_dir = "outputs/sticker_fid/real"
os.makedirs(generated_dir, exist_ok=True)
os.makedirs(real_dir, exist_ok=True)

print("🖼 Generating validation images...")
for i, sample in enumerate(tqdm(val_set)):
    prompt = sample["prompt"]
    real_image_tensor = sample["pixel_values"]
    
    # Save real image
    real_image = transforms.ToPILImage()(real_image_tensor)
    real_image.save(os.path.join(real_dir, f"real_{i:04}.png"))

    # Generate and save fake image
    result = pipe(prompt, num_inference_steps=50, guidance_scale=7.5, height=256, width=256)
    gen_image = result.images[0]
    gen_image.save(os.path.join(generated_dir, f"gen_{i:04}.png"))

# Compute FID using pytorch_fid
print("📊 Calculating FID...")
fid_score = calculate_fid_given_paths([real_dir, generated_dir], batch_size=32, device=device, dims=2048)
print(f" FID Score: {fid_score:.4f}")


Using device: cuda


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

 LoRA model loaded.
🖼 Generating validation images...


  0%|          | 0/44 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

100%|██████████| 44/44 [04:31<00:00,  6.18s/it]


📊 Calculating FID...


100%|██████████| 2/2 [00:03<00:00,  1.92s/it]


 FID Score: 240.1287
